In [ ]:
%matplotlib inline
import os
import numpy
import timeit
import matplotlib.pyplot as plt
from IPython.display import Image
from scipy import ndimage
from sklearn.preprocessing import Imputer

In [ ]:
test_path = "../captures/testing/IMG_2114.PNG"
Image(filename=test_path)

In [ ]:
combined_image = ndimage.imread(test_path)
combined_image.shape

In [ ]:
def split(image):
    split_height = image.shape[0] / 2
    return image[:split_height], image[split_height:]
camera_image, depth_image = split(combined_image)
print camera_image.shape, depth_image.shape

In [ ]:
depth_image[0].shape

In [ ]:
def convert_to_depth(image):
    height = image.shape[0]
    width = image.shape[1]
    depths = numpy.zeros((height, width), dtype=numpy.float32)
    BYTE_MAX = 255
    CHANNEL_MAX = 8
    MAX_RED_VALUE = BYTE_MAX - CHANNEL_MAX
    CHANNELS_MAX = CHANNEL_MAX * CHANNEL_MAX
    INVALID = float('Nan')
    found_orientation = False
    orientation = [1, 0, 0, 0]
    for y in xrange(0, height):
        row = image[y]
        depth_row = depths[y]
        for x in xrange(0, width):
            pixel = row[x]
            red = pixel[0]
            if x == 0 and y == 0 and red == BYTE_MAX:
                found_orientation = True
                depth_row[x] = INVALID
                continue
            if found_orientation:
                for c in range(0, len(orientation)):
                    orientation[c] = ((2.0 * pixel[c]) / BYTE_MAX) - 1
                found_orientation = False
                depth_row[x] = INVALID
                continue
            if red == 0:
                depth_row[x] = INVALID
                continue
            depth_row[x] = (MAX_RED_VALUE - red) * CHANNELS_MAX + ((pixel[1] - red) * CHANNEL_MAX) + (pixel[2] - red)
    return depths, orientation

In [ ]:
depths, attitude = convert_to_depth(depth_image)

In [ ]:
depths.shape

In [ ]:
attitude

In [ ]:
depths[0][0:4]

In [ ]:
depths[120][300:310]

In [ ]:
def decode_depth(image):
    BYTE_MAX = 255
    CHANNEL_MAX = 8.0
    MAX_RED_VALUE = BYTE_MAX - CHANNEL_MAX
    CHANNELS_MAX = CHANNEL_MAX * CHANNEL_MAX
    orientation = [1, 0, 0, 0]
    
    if numpy.array_equal(image[0, 0], [BYTE_MAX, 0, 0, BYTE_MAX]):
        print "Found attitude"
        pixel = image[0, 1]
        for c in range(len(orientation)):
            orientation[c] = ((2.0 * pixel[c]) / BYTE_MAX) - 1
        image[0, 0] = [0, 0, 0, BYTE_MAX]
        image[0, 1] = [0, 0, 0, BYTE_MAX]
            

    red = image[:, :, 0]
    green = image[:, :, 1]
    blue = image[:, :, 2]

    depth = ((MAX_RED_VALUE - red) * CHANNELS_MAX) + ((green - red) * CHANNEL_MAX) + (blue - red)
    depth[numpy.where(red == 0)] = float('nan')
    return depth, orientation

In [ ]:
depths, attitude = decode_depth(depth_image)

In [ ]:
depths.shape

In [ ]:
depths[120][300:310]

In [ ]:
attitude

In [ ]:
timeit.timeit("depths, attitude = decode_depth(depth_image)", number=10, setup="from __main__ import decode_depth,depth_image")

In [ ]:
timeit.timeit("depths, attitude = convert_to_depth(depth_image)", number=10, setup="from __main__ import convert_to_depth,depth_image")

In [ ]:
plt.imshow(depths)

In [ ]:
imp = Imputer(strategy="max")
cleaned = imp.fit_transform(depths)

In [ ]:
plt.imshow(cleaned)

In [ ]:
depths[330,0]

In [ ]:
cleaned[330,0]